In [112]:
import os
import re
import pandas as pd

In [113]:
import spacy
nlp = spacy.load("pt_core_news_sm")

# Limpeza dos discursos

## 1 Criando uma lista com todos os discursos

In [114]:
# Recebendo os discursos
discursos = []
for j in os.listdir('discursos'):
    file = open(f'discursos/{j}', 'r', encoding="utf-8")
    text = file.read()
    discursos.append(text)
    file.close  


In [115]:
discursos = pd.Series(discursos)
discursos

0     [A SRA. CHRISTIANE DE SOUZA YARED ]+[(Bloco/PP...
1     [A SRA. SORAYA SANTOS ]+[(PL - RJ. Pela ordem....
2     [O SR. MÁRCIO MACÊDO ]+[(Bloco/PT - SE. Pela L...
3     [O SR. TIAGO MITRAUD ]+[(NOVO - MG. Para discu...
4     [O SR. BIRA DO PINDARÉ ]+[(Bloco/PSB - MA. Pel...
                            ...                        
95    [O SR. AGUINALDO RIBEIRO ]+[(Bloco/PP - PB. Pe...
96    [O SR. RICARDO BARROS ]+[(Bloco/PP - PR. Pela ...
97    [O SR. DANIEL SILVEIRA ]+[(UNIÃO - RJ. Pela or...
98    [O SR. IVAN VALENTE ]+[(PSOL - SP. Pela Lidera...
99    [O SR. WELITON PRADO ]+[(Bloco/PROS - MG. Pela...
Length: 100, dtype: object

In [116]:
# #Criando lista com todos os conectivos da lingua portuguesa
# conectivos = pd.read_html('https://duvidas.dicio.com.br/conectivos/')

# a = conectivos[0]['Exemplos:']
# conectivos_lista = []
# for j in a:
#     conectivos_lista.append(j.upper())
# conectivos_lista = " ".join(conectivos_lista)

# conectivos_lista = conectivos_lista.replace('; ', ' ').replace('… ', ' ').split(' ')
# conectivos_lista = list(dict.fromkeys(conectivos_lista))
# outros_conectivos = ['OS', 'E', 'UM', 'UMA']
# for i in outros_conectivos:
#     conectivos_lista.append(i)
# conectivos_lista = sorted(conectivos_lista)

## 2 Verificando se houve revisão nos discursos

In [117]:
revisado = []
for j in range(len(discursos)):
    if(re.search(r'Sem revisão', discursos[j])):
        revisado.append(False)
    else:
        revisado.append(True)
revisao = pd.Series(revisado)

In [118]:
df_discursos = {"discursos" : discursos,
                "revisao" : revisao}
df_discursos = pd.concat(df_discursos, axis=1)

## 3 Limpando os discurso

In [119]:
from stop_words import get_stop_words
conectivos_lista = []
conectivos_lista = get_stop_words('pt')
conectivos_lista.append('é')
conectivos_lista = [conectivo.upper() for conectivo in conectivos_lista]

In [120]:
#Retirando a introdução
discursos = df_discursos['discursos'].str.split('.\) -')
df_discursos['discursos'] = [i[1:] for i in discursos]

discursos_lista = []
discurso_limpo = []

for i in df_discursos['discursos']:
    discursos_lista.append(" ".join(i))

termos_discursos = []
for i in range(len(discursos_lista)):

    #Retirando os sons ambientes
    sons_ambiente = ['<tr><td align="center">(<i>Soa a campainha.</i>)</td></tr></table>', '<i>(Palmas.)</i>']
    for j in sons_ambiente:
        discursos_lista[i] = discursos_lista[i].replace(j, '')
    
    #Retirando as tags
    tags = ['<i>', '</i>', f'\n']
    for j in tags:
        discursos_lista[i] = discursos_lista[i].replace(j, '')

    #Retirando pontuacoes
    pontuacoes = ['.', ':', '...', '(', ')', '[', ']', '{', '}', '!', '?', ',', ';', '-', '_', "'", "\"", "nº", '<', '>', '\\', '|', '/', ', , ', ']\n']
    for j in pontuacoes:
        discursos_lista[i] = discursos_lista[i].replace(j, '')

    #retirando numeros
    for j in range(10):
        discursos_lista[i] = discursos_lista[i].replace(str(j), '')

    discurso_limpo.append(discursos_lista[i])
    discursos_lista[i] = discursos_lista[i].split(' ')
    
    #separando os termos e retirando conectivos
    termos = []
    for j in range(len(discursos_lista[i])):
        if (discursos_lista[i][j] != '') and (type(discursos_lista[i][j]) == str):
            termos.append(discursos_lista[i][j].lower())
    termos = [termo for termo in termos if not termo in conectivos_lista]
    termos_discursos.append(termos)


df_discursos['discursos'] = discurso_limpo




In [121]:
len(termos_discursos[0][1])

2

In [128]:
token_terms = []

for i in range(len(termos_discursos)):

    terms = []
    for j in range(len(termos_discursos[i])):
        word = nlp(termos_discursos[i][j])
        for token in word:
            if token.tag_ != 'NPROP'or token.tag_ == 'VERB':
                terms.append(token.text)
                print(token.text, "|", token.tag_, "|", spacy.explain(token.tag_))
    
    #Aplicando a lemmatizatição

    lemma_list = []
    for j in range(len(terms)):
        word = nlp(terms[j])
        for token in word:
            lemma_list.append(token.lemma_)


    terms = [termo for termo in lemma_list]
    token_terms.append(terms)


df_discursos['termos'] = [pd.Series((i).upper()) for i in token_terms]

obrigada | ADJ | adjective
sr | PROPN | proper noun
presidente | NOUN | noun
sr | PROPN | proper noun
presidente | NOUN | noun
eu | PRON | pronoun
quero | VERB | verb
aqui | ADV | adverb
pedir | VERB | verb
a | ADP | adposition
todos | PRON | pronoun
os | DET | determiner
colegas | NOUN | noun
os | DET | determiner
nossos | DET | determiner
amigos | NOUN | noun
deputados | NOUN | noun
e | CCONJ | coordinating conjunction
senadores | NOUN | noun
também | ADV | adverb
que | SCONJ | subordinating conjunction
nos | ADP | adposition
permitam | VERB | verb
fazer | VERB | verb
com | ADP | adposition
que | SCONJ | subordinating conjunction
vocês | PRON | pronoun
entendam | VERB | verb
qual | PRON | pronoun
é | AUX | auxiliary
a | ADP | adposition
nossa | DET | determiner
preocupação | NOUN | noun
com | ADP | adposition
nós | PRON | pronoun
não | ADV | adverb
derrubarmos | VERB | verb
o | DET | determiner
veto | NOUN | noun
é | AUX | auxiliary
muito | ADV | adverb
sério | ADJ | adjective
o | DE

AttributeError: 'list' object has no attribute 'upper'

In [125]:
df_discursos['termos'][0]

0          obrigado
1            querer
2             pedir
3          permitir
4             fazer
5          entendar
6         derrubarr
7             sério
8            passar
9               ter
10         infinito
11    irresponsável
12          dirigir
13         alcançar
14           filmar
15           causar
16           grande
17              ter
18         arrombar
19           correr
20         arrombar
21           querer
22           evitar
23            ficar
24         desolada
25           perdir
26    irresponsável
27       necessário
28            venho
29            pedir
30        conclamar
31          colocar
32         possível
33         assistar
34            ficar
35         indignar
36          ocorrer
37            pegar
38           correr
39        atropelar
40              ter
41         precisar
42            punir
43    irresponsável
44         precisar
45            fazer
46            poder
47            tirar
48      compreender
49            peçar


In [126]:
df_discursos

,discursos,revisao,termos
0,Obrigada Sr Presidente Sr Presidente eu quer...,False,0 obrigado 1 querer 2 ...
1,Presidente eu queria primeiro me dirigir a V ...,False,0 querer 1 primeiro 2 dirigir 3...
2,Sr Presidente Marcelo Ramos Sras Deputadas e ...,False,0 deputada 1 utilizar 2 fala...
3,Perfeito Obrigado Presidente Sei que o acor...,False,0 perfeito 1 obrigar 2 s...
4,Sr Presidente Sras e Srs Parlamentares eu que...,False,0 querer 1 registrar 2 c...
...,...,...,...
95,Sr Presidente Deputado Marcelo eu queria prim...,False,0 querer 1 primeiro 2 ...
96,Obrigado Presidente Peço também o esforço de...,False,0 obrigar 1 peçar 2 con...
97,Presidente obrigado Eu quero só notificar a...,False,0 obrigar 1 querer 2 ...
98,Obrigado Sr Presidente Sras e Srs Parlamenta...,False,0 obrigar 1 querer 2 primeir...


## 4 Contagem de termos

In [ ]:
'''qnt_termo = []
for i in range(len(df_discursos)):
    qnt_termo.append(df_discursos['termos'][i].value_counts().reset_index().rename(columns = {'index' : 'termo', 0 : 'qnt_termo'}))
df_discursos['qnt_termo'] = qnt_termo'''

"qnt_termo = []\nfor i in range(len(df_discursos)):\n    qnt_termo.append(df_discursos['termos'][i].value_counts().reset_index().rename(columns = {'index' : 'termo', 0 : 'qnt_termo'}))\ndf_discursos['qnt_termo'] = qnt_termo"

In [ ]:
dict_termos = []
for i in range(len(df_discursos['termos'])):
    new_dict = {df_discursos['termos'][i].value_counts().index[j]: df_discursos['termos'][i].value_counts()[j] for j in range(len(df_discursos['termos'][i].value_counts()))}
    dict_termos.append(new_dict)

In [ ]:
df_discursos['qtd_termos'] = dict_termos
df_discursos

,discursos,revisao,termos,qtd_termos
0,Obrigada Sr Presidente Sr Presidente eu quer...,False,0 OBRIGADA 1 SR 2 PRE...,"{'VETO': 4, 'MORTES': 3, 'TODOS': 3, 'IRRESPON..."
1,Presidente eu queria primeiro me dirigir a V ...,False,0 PRESIDENTE 1 QUERIA 2 P...,"{'V': 8, 'EXA': 8, 'VETO': 7, 'EXPRESSÃO': 5, ..."
2,Sr Presidente Marcelo Ramos Sras Deputadas e ...,False,0 SR 1 PRESIDENTE 2 ...,"{'SR': 12, 'BRASIL': 12, 'HOJE': 12, 'POVO': 1..."
3,Perfeito Obrigado Presidente Sei que o acor...,False,0 PERFEITO 1 OBRIGADO 2 PRE...,"{'NOVO': 5, 'AQUI': 4, 'ACORDO': 4, 'CONSTITUC..."
4,Sr Presidente Sras e Srs Parlamentares eu que...,False,0 SR 1 PRESIDENTE 2 ...,"{'VAMOS': 9, 'PARTIDO': 7, 'BRASIL': 6, 'PRESI..."
...,...,...,...,...
95,Sr Presidente Deputado Marcelo eu queria prim...,False,0 SR 1 PRESIDENTE 2 D...,"{'EXA': 6, 'ACORDO': 6, 'V': 6, 'TEMPO': 5, 'S..."
96,Obrigado Presidente Peço também o esforço de...,False,0 OBRIGADO 1 PRESIDENTE 2 ...,"{'PEÇO': 3, 'TODOS': 3, 'ACORDO': 3, 'QUESTÕES..."
97,Presidente obrigado Eu quero só notificar a...,False,0 PRESIDENTE 1 OBRIGADO 2 ...,"{'PRESIDENTE': 4, 'CASA': 4, 'DENTRO': 3, 'AQU..."
98,Obrigado Sr Presidente Sras e Srs Parlamenta...,False,0 OBRIGADO 1 SR 2 PRE...,"{'AQUI': 13, 'QUER': 10, 'FEDERAL': 8, 'VAMOS'..."


In [ ]:
df_discursos['termos'][0]

0        OBRIGADA
1              SR
2      PRESIDENTE
3              SR
4      PRESIDENTE
          ...    
121         ATRÁS
122       VOLANTE
123          PEÇO
124     DERRUBADA
125          VETO
Length: 126, dtype: object

In [ ]:
df_discursos['termos'] = [df_discursos['termos'][i].tolist() for i in range(len(df_discursos['termos']))]

In [ ]:
df_discursos.to_csv('df_discursos.csv', encoding='utf-8', index=False)

In [ ]:
a = pd.read_csv('df_discursos.csv')
a

,discursos,revisao,termos,qtd_termos
0,Obrigada Sr Presidente Sr Presidente eu quer...,False,"['OBRIGADA', 'SR', 'PRESIDENTE', 'SR', 'PRESID...","{'VETO': 4, 'MORTES': 3, 'TODOS': 3, 'IRRESPON..."
1,Presidente eu queria primeiro me dirigir a V ...,False,"['PRESIDENTE', 'QUERIA', 'PRIMEIRO', 'DIRIGIR'...","{'V': 8, 'EXA': 8, 'VETO': 7, 'EXPRESSÃO': 5, ..."
2,Sr Presidente Marcelo Ramos Sras Deputadas e ...,False,"['SR', 'PRESIDENTE', 'MARCELO', 'RAMOS', 'SRAS...","{'SR': 12, 'BRASIL': 12, 'HOJE': 12, 'POVO': 1..."
3,Perfeito Obrigado Presidente Sei que o acor...,False,"['PERFEITO', 'OBRIGADO', 'PRESIDENTE', 'SEI', ...","{'NOVO': 5, 'AQUI': 4, 'ACORDO': 4, 'CONSTITUC..."
4,Sr Presidente Sras e Srs Parlamentares eu que...,False,"['SR', 'PRESIDENTE', 'SRAS', 'SRS', 'PARLAMENT...","{'VAMOS': 9, 'PARTIDO': 7, 'BRASIL': 6, 'PRESI..."
...,...,...,...,...
95,Sr Presidente Deputado Marcelo eu queria prim...,False,"['SR', 'PRESIDENTE', 'DEPUTADO', 'MARCELO', 'Q...","{'EXA': 6, 'ACORDO': 6, 'V': 6, 'TEMPO': 5, 'S..."
96,Obrigado Presidente Peço também o esforço de...,False,"['OBRIGADO', 'PRESIDENTE', 'PEÇO', 'ESFORÇO', ...","{'PEÇO': 3, 'TODOS': 3, 'ACORDO': 3, 'QUESTÕES..."
97,Presidente obrigado Eu quero só notificar a...,False,"['PRESIDENTE', 'OBRIGADO', 'QUERO', 'NOTIFICAR...","{'PRESIDENTE': 4, 'CASA': 4, 'DENTRO': 3, 'AQU..."
98,Obrigado Sr Presidente Sras e Srs Parlamenta...,False,"['OBRIGADO', 'SR', 'PRESIDENTE', 'SRAS', 'SRS'...","{'AQUI': 13, 'QUER': 10, 'FEDERAL': 8, 'VAMOS'..."
